<a href="https://colab.research.google.com/github/ikhsan-muhammad/p01-airplane_cargo/blob/main/p01-airplane-cargo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Airplane Cargo Analysist

<img src="https://indianonlineseller.com/images/blogs/editor/amazon-transportation-services-helping-hand-or-grabbing-hand.jpg" alt="Flowers in Chania" style="border-radius: 15px 15px 15px 15px;"/>

**Metadata:**

https://data.sfgov.org/Transportation/Air-Traffic-Cargo-Statistics/u397-j8nr/about_data [san francisco air cargo data]

# Understanding Problem

**Problem case:**
* Evaluate the performance of different airlines based on cargo metrics, identifying top performers and areas for improvement.
* Analyze patterns in different types of cargo activities to understand trends in the industry, such as the prevalence of express shipments or bulk cargo.
* Examine cargo operations over time to identify seasonality, peak periods, or long-term trends in air cargo demand.
* Benchmark the performance of specific airlines, regions, or aircraft types against industry averages to assess competitiveness
* Utilize historical data to build predictive models for future air cargo trends, helping stakeholders make informed decisions.

# Set-UP

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# # pull data from kaggle datasets
# ! pip install kaggle
# ! mkdir ~/.kaggle
# ! cp /content/drive/MyDrive/Colab\ Notebooks/kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download ikhsanmhmd/air-traffic-cargo-statistic
# ! unzip /content/air-traffic-cargo-statistic.zip
# ! rm /content/air-traffic-cargo-statistic.zip

# Data Wrangling

## Import

In [3]:
#import necessary library
import pandas as pd
pd.plotting.register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import nbformat as nb

#set all things that needed
pd.set_option('display.max_columns', 100)
# pd.set_option('display.max_rows',200)
pd.reset_option('display.max_rows')
# plt.style.use('dark_background')

# with kaggle
# df = pd.read_csv('/kaggle/input/air-traffic-cargo-statistic/Air_Traffic_Cargo_Statistics.csv', parse_dates=[1,13,14], date_format='mixed')
# with colab
# df = pd.read_csv('/content/drive/MyDrive/Ori Data Import/Air_Traffic_Cargo_Statistics.csv', parse_dates=[1,13,14], date_format='mixed')
# local
df = pd.read_csv('D:/.project/p01-airplane_cargo/Air_Traffic_Cargo_Statistics.csv', parse_dates=[1,13,14], date_format='mixed')

In [4]:
# df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55038 entries, 0 to 55037
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Activity Period              55038 non-null  int64         
 1   Activity Period Start Date   55038 non-null  datetime64[ns]
 2   Operating Airline            55038 non-null  object        
 3   Operating Airline IATA Code  54460 non-null  object        
 4   Published Airline            55038 non-null  object        
 5   Published Airline IATA Code  54460 non-null  object        
 6   GEO Summary                  55038 non-null  object        
 7   GEO Region                   55038 non-null  object        
 8   Activity Type Code           55038 non-null  object        
 9   Cargo Type Code              55038 non-null  object        
 10  Cargo Aircraft Type          55038 non-null  object        
 11  Cargo Weight LBS             55038 non-nu

In [5]:
df.head()

,Activity Period,Activity Period Start Date,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Cargo Type Code,Cargo Aircraft Type,Cargo Weight LBS,Cargo Metric TONS,data_as_of,data_loaded_at
0,199907,1999-07-01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Deplaned,Cargo,Passenger,349048,158.328173,2024-06-20 13:00:06,2024-06-22 15:01:45
1,199907,1999-07-01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Deplaned,Mail,Passenger,122003,55.340561,2024-06-20 13:00:06,2024-06-22 15:01:45
2,199907,1999-07-01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Enplaned,Cargo,Passenger,514450,233.354520,2024-06-20 13:00:06,2024-06-22 15:01:45
3,199907,1999-07-01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Enplaned,Mail,Passenger,89776,40.722394,2024-06-20 13:00:06,2024-06-22 15:01:45
4,199907,1999-07-01,Aeroflot Russian International Airlines,NaN,Aeroflot Russian International Airlines,NaN,International,Europe,Deplaned,Cargo,Passenger,13086,5.935810,2024-06-20 13:00:06,2024-06-22 15:01:45


In [6]:
#select all columns that we need
selected_df = df[[
#     'Activity Period',
    'Activity Period Start Date',
    'Operating Airline',
#     'Operating Airline IATA Code',
    'Published Airline',
#     'Published Airline IATA Code',
    'GEO Summary',
    'GEO Region',
    'Activity Type Code',
    'Cargo Type Code',
    'Cargo Aircraft Type',
    'Cargo Weight LBS',
    # 'Cargo Metric TONS',
#     'data_as_of',
#     'data_loaded_at'
]].copy()

selected_df

,Activity Period Start Date,Operating Airline,Published Airline,GEO Summary,GEO Region,Activity Type Code,Cargo Type Code,Cargo Aircraft Type,Cargo Weight LBS
0,1999-07-01,ATA Airlines,ATA Airlines,Domestic,US,Deplaned,Cargo,Passenger,349048
1,1999-07-01,ATA Airlines,ATA Airlines,Domestic,US,Deplaned,Mail,Passenger,122003
2,1999-07-01,ATA Airlines,ATA Airlines,Domestic,US,Enplaned,Cargo,Passenger,514450
3,1999-07-01,ATA Airlines,ATA Airlines,Domestic,US,Enplaned,Mail,Passenger,89776
4,1999-07-01,Aeroflot Russian International Airlines,Aeroflot Russian International Airlines,International,Europe,Deplaned,Cargo,Passenger,13086
...,...,...,...,...,...,...,...,...,...
55033,2024-04-01,Vietnam Airlines JSC,Vietnam Airlines JSC,International,Asia,Enplaned,Cargo,Passenger,19323
55034,2024-04-01,Virgin Atlantic,Virgin Atlantic,International,Europe,Deplaned,Cargo,Passenger,414753
55035,2024-04-01,Virgin Atlantic,Virgin Atlantic,International,Europe,Enplaned,Cargo,Passenger,174269
55036,2024-04-01,ZIPAIR Tokyo Inc,ZIPAIR Tokyo Inc,International,Asia,Deplaned,Cargo,Passenger,845137


In [7]:
#change all columns name
selected_df = selected_df.rename(columns={
    'Activity Period Start Date' : 'period',
    'Operating Airline' : 'operating_airline',
    'Published Airline' : 'published_airline',
    'GEO Summary' : 'geo_summary',
    'GEO Region' : 'geo_region',
    'Activity Type Code' : 'activity_type',
    'Cargo Type Code' : 'cargo_type',
    'Cargo Aircraft Type' : 'aircraft_type',
    'Cargo Weight LBS' : 'lbs'
}) # type: ignore

selected_df

,period,operating_airline,published_airline,geo_summary,geo_region,activity_type,cargo_type,aircraft_type,lbs
0,1999-07-01,ATA Airlines,ATA Airlines,Domestic,US,Deplaned,Cargo,Passenger,349048
1,1999-07-01,ATA Airlines,ATA Airlines,Domestic,US,Deplaned,Mail,Passenger,122003
2,1999-07-01,ATA Airlines,ATA Airlines,Domestic,US,Enplaned,Cargo,Passenger,514450
3,1999-07-01,ATA Airlines,ATA Airlines,Domestic,US,Enplaned,Mail,Passenger,89776
4,1999-07-01,Aeroflot Russian International Airlines,Aeroflot Russian International Airlines,International,Europe,Deplaned,Cargo,Passenger,13086
...,...,...,...,...,...,...,...,...,...
55033,2024-04-01,Vietnam Airlines JSC,Vietnam Airlines JSC,International,Asia,Enplaned,Cargo,Passenger,19323
55034,2024-04-01,Virgin Atlantic,Virgin Atlantic,International,Europe,Deplaned,Cargo,Passenger,414753
55035,2024-04-01,Virgin Atlantic,Virgin Atlantic,International,Europe,Enplaned,Cargo,Passenger,174269
55036,2024-04-01,ZIPAIR Tokyo Inc,ZIPAIR Tokyo Inc,International,Asia,Deplaned,Cargo,Passenger,845137


In [8]:
#Always run this cell one time, cant do it twice
selected_df['period'] = selected_df['period'].dt.to_period('M')
selected_df

,period,operating_airline,published_airline,geo_summary,geo_region,activity_type,cargo_type,aircraft_type,lbs
0,1999-07,ATA Airlines,ATA Airlines,Domestic,US,Deplaned,Cargo,Passenger,349048
1,1999-07,ATA Airlines,ATA Airlines,Domestic,US,Deplaned,Mail,Passenger,122003
2,1999-07,ATA Airlines,ATA Airlines,Domestic,US,Enplaned,Cargo,Passenger,514450
3,1999-07,ATA Airlines,ATA Airlines,Domestic,US,Enplaned,Mail,Passenger,89776
4,1999-07,Aeroflot Russian International Airlines,Aeroflot Russian International Airlines,International,Europe,Deplaned,Cargo,Passenger,13086
...,...,...,...,...,...,...,...,...,...
55033,2024-04,Vietnam Airlines JSC,Vietnam Airlines JSC,International,Asia,Enplaned,Cargo,Passenger,19323
55034,2024-04,Virgin Atlantic,Virgin Atlantic,International,Europe,Deplaned,Cargo,Passenger,414753
55035,2024-04,Virgin Atlantic,Virgin Atlantic,International,Europe,Enplaned,Cargo,Passenger,174269
55036,2024-04,ZIPAIR Tokyo Inc,ZIPAIR Tokyo Inc,International,Asia,Deplaned,Cargo,Passenger,845137


In [9]:
selected_df['operating_airline'].value_counts().sort_index()

operating_airline
21 Air, LLC             28
ABX Air                346
ASTAR USA, LLC          16
ATA Airlines           324
Active Aero Charter      2
                      ... 
Virgin America           5
Virgin Atlantic        718
WOW Air                 55
World Airways           36
ZIPAIR Tokyo Inc        17
Name: count, Length: 132, dtype: int64

In [10]:
#see all airline names contain (,)
selected_df['operating_airline'].loc[selected_df['operating_airline'].str.contains(',')].value_counts().index

Index(['All Nippon Company Airways, Ltd.', 'TACA International Airlines, S.A.',
       'Aer Lingus, Ltd.', 'Atlas Air, Inc', 'COPA Airlines, Inc.',
       'Air Transport International, Inc.', 'Atlas Air, Inc. (inactive)',
       'China Eastern Airlines, Inc', 'Emery Worldwide Airlines, Inc.',
       '21 Air, LLC', 'ASTAR USA, LLC', 'Air Transport International, LLC.',
       'Total Airport Services, Inc. (Inactive)', 'Norse Atlantic UK, Ltd.',
       'Envoy Air, Inc.'],
      dtype='object', name='operating_airline')

In [11]:
#remove extension from airlines name
def airline_name_cleaner(selected_df):
    selected_df = selected_df.replace(r'\bInc\b|\bLLC\b|\bLimited\b|\bLtd\b|\binactive\b|\bPre 07/01/2013\b|\bbecame Delta\b|\bLTD\b|\bGmbH\b|\bEHF\b|\bASA\b|\bCO\b|\bJSC\b|\bSPA\b|\bSA\b', '',regex=True).copy()
    selected_df = selected_df.replace(r'\([^)]*\)|[-,.]', '', regex=True)
    selected_df = selected_df.map(lambda x: x.strip() if isinstance(x, str) else x) #remove all white space
    return selected_df

selected_df = airline_name_cleaner(selected_df)
selected_df['operating_airline'].value_counts().sort_index()

operating_airline
21 Air                  28
ABX Air                346
ASTAR USA               16
ATA Airlines           324
Active Aero Charter      2
                      ... 
Virgin America           5
Virgin Atlantic        718
WOW Air                 55
World Airways           36
ZIPAIR Tokyo            17
Name: count, Length: 128, dtype: int64

In [12]:
selected_df['operating_airline'].value_counts().sort_index()

operating_airline
21 Air                  28
ABX Air                346
ASTAR USA               16
ATA Airlines           324
Active Aero Charter      2
                      ... 
Virgin America           5
Virgin Atlantic        718
WOW Air                 55
World Airways           36
ZIPAIR Tokyo            17
Name: count, Length: 128, dtype: int64

In [13]:
selected_df.describe()

,lbs
count,5.503800e+04
mean,5.204636e+05
std,1.023128e+06
min,1.000000e+00
25%,1.936650e+04
50%,1.442165e+05
75%,5.915170e+05
max,2.381234e+07


In [14]:
selected_df.describe(include='object')

,operating_airline,published_airline,geo_summary,geo_region,activity_type,cargo_type,aircraft_type
count,55038,55038,55038,55038,55038,55038,55038
unique,128,116,2,9,2,3,3
top,United Airlines,United Airlines,International,Asia,Deplaned,Cargo,Passenger
freq,9455,9476,38475,19314,28117,30675,45285


In [15]:
selected_df['period'] = selected_df['period'].dt.to_timestamp()

## Exploratory Data Analysis (EDA)

**Problem Case**
1. Between 1999 to 2024, which airline get increased air traffic cargo? Which get less air traffic cargo?
2. Is there any distinct characteristics between domestic and international flight?
3. What aircraft type often fly internationally?
4. Describe the cargo weight variable and its influence across the data

In [16]:
selected_df['year'] = selected_df.period.dt.year
selected_df['month'] = selected_df.period.dt.month
selected_df

,period,operating_airline,published_airline,geo_summary,geo_region,activity_type,cargo_type,aircraft_type,lbs,year,month
0,1999-07-01,ATA Airlines,ATA Airlines,Domestic,US,Deplaned,Cargo,Passenger,349048,1999,7
1,1999-07-01,ATA Airlines,ATA Airlines,Domestic,US,Deplaned,Mail,Passenger,122003,1999,7
2,1999-07-01,ATA Airlines,ATA Airlines,Domestic,US,Enplaned,Cargo,Passenger,514450,1999,7
3,1999-07-01,ATA Airlines,ATA Airlines,Domestic,US,Enplaned,Mail,Passenger,89776,1999,7
4,1999-07-01,Aeroflot Russian International Airlines,Aeroflot Russian International Airlines,International,Europe,Deplaned,Cargo,Passenger,13086,1999,7
...,...,...,...,...,...,...,...,...,...,...,...
55033,2024-04-01,Vietnam Airlines,Vietnam Airlines,International,Asia,Enplaned,Cargo,Passenger,19323,2024,4
55034,2024-04-01,Virgin Atlantic,Virgin Atlantic,International,Europe,Deplaned,Cargo,Passenger,414753,2024,4
55035,2024-04-01,Virgin Atlantic,Virgin Atlantic,International,Europe,Enplaned,Cargo,Passenger,174269,2024,4
55036,2024-04-01,ZIPAIR Tokyo,ZIPAIR Tokyo,International,Asia,Deplaned,Cargo,Passenger,845137,2024,4


### Case 1

In [ ]:
# Case 1 
## operation_check = selected_df.groupby([selected_df.period.dt.year, 'operating_airline'])['lbs'].sum()
pivot_data = selected_df.pivot_table(index='operating_airline', 
                        columns='year',
                        values='lbs',   
                        aggfunc='sum')
pivot_data = pivot_data.reset_index()


In [45]:
pivot_data = pivot_data[pivot_data.notna().all(axis=1)]

In [44]:

# plot line 1
# monthly_traffic = selected_df.groupby(['period','operating_airline'])['lbs'].sum().reset_index()
yearly_traffic = selected_df.groupby([selected_df.period.dt.year,'operating_airline'])['lbs'].sum().reset_index()

px.line(yearly_traffic,
        x='period',
        y='lbs',
        color='operating_airline',
        title='Tren Cargo (lbs) per Airline per year for All Airlines',
        markers=True).show()

# plot line 2
pivot_table_melted = pivot_data.melt(id_vars=['operating_airline'], 
                                            var_name='year', 
                                            value_name='total_lbs')

px.line(pivot_table_melted, 
              x='year', 
              y='total_lbs', 
              color='operating_airline', 
              labels={'total_lbs':'Total lbs', 'year':'Year', 'operating_airline':'Airline'},
              title='Tren Cargo (lbs) per Airline per year with Active Airlines',
              markers=True).show()

### Case 2